#This is the wrangling code when updatng the library data using data from https://data.gov.au/dataset/ds-vic-77d8ec1d-09f1-4b12-a50a-46ed662b980b/details?q=library%20victoria.

In [1]:
#https://towardsdatascience.com/reverse-geocoding-in-python-a915acf29eb6
#install packages
!pip install ipython-autotime
!pip install geopandas
!pip install geopy
%load_ext autotime

In [2]:
#all libraries used
import pandas as pd
import numpy as np
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim

#instantiate nominatim as the locator to be used for this notebook

time: 114 ms


**This section is simulate the data wrangling for libraries.csv**

In [3]:
#input file paths for the libraries (.csv file) and the database (.csv file)
#change lib_path value path of libraries .csv file as downloaded in https://data.gov.au/dataset/ds-vic-77d8ec1d-09f1-4b12-a50a-46ed662b980b/details?q=library%20victoria
lib_path = "/content/drive/My Drive/B3_FIT5120FIT5122/Iteration 1/Dataset/Raw Data Set/libraries.csv"

#change db_path value to path of database .csv file
#for csv files with separator is comma ","
#db_path = "/content/drive/My Drive/B3_FIT5120FIT5122/Iteration 2/Dataset/For Upload/05142020_upload_v1.csv"

#for csv files with separator is semi colon ";"
db_path = "/content/drive/My Drive/B3_FIT5120FIT5122/Iteration 2/Dataset/Python Code/stations.csv"

#create db columns
cols_db = ['id','station_name', 'longitude', 'latitude', 'address', 'suburb', 'postcode', 'if_charger_working', 'usb_a', 'usb_c', 'micro_usb', 'plug_only', 'establishment_type', 'if_wifi', 'if_bathroom', 'access_type', 'other_amenities', 'star_rating', 'mon_open', 'mon_close', 'tue_open', 'tue_close', 'wed_open', 'wed_close', 'thu_open', 'thu_close', 'fri_open', 'fri_close', 'sat_open', 'sat_close', 'sun_open', 'sun_close', 'if_24h']

#create list of postcodes in melbourne, avoid reading a separate file
melb_postcode = [3053, 3053, 3054, 3008, 3002, 3002, 3031, 3031, 3000, 3004, 3051, 3051, 3051, 3052, 3052, 3207, 3207, 3006, 3006, 3141, 3003, 3003, 3206, 3183, 3184, 3184, 3004, 3004, 3206, 3207, 3207, 3207, 3207, 3207, 3207, 3185, 3182, 3182, 3183, 3182, 3182, 3006, 3205, 3205, 3205, 3205, 3006, 3067, 3067, 3078, 3121, 3054, 3054, 3068, 3066, 3066, 3121, 3121, 3078, 3078, 3065, 3065, 3065, 3068, 3068, 3068, 3054, 3121, 3121, 3121, 3121, 3121, 3121, 3081, 3088, 3083, 3083, 3083, 3083, 3083, 3084, 3095, 3095, 3088, 3088, 3088, 3088, 3084, 3081, 3081, 3081, 3079, 3079, 3079, 3079, 3079, 3093, 3085, 3085, 3085, 3094, 3084, 3084, 3088, 3084, 3084, 3087, 3087, 3085, 3078, 3083, 3083, 3078, 3083, 3085, 3070, 3070, 3070, 3070, 3070, 3070, 3070, 3070, 3072, 3072, 3072, 3072, 3072, 3072, 3072, 3072, 3072, 3072, 3072, 3073, 3073, 3073, 3073, 3073, 3073, 3073, 3073, 3071, 3071, 3071, 3049, 3047, 3047, 3047, 3047, 3047, 3061, 3048, 3064, 3047, 3047, 3043, 3059, 3047, 3036, 3048, 3045, 3064, 3062, 3043, 3049, 3429, 3429, 3429, 3429, 3428, 3430, 3430, 3427, 3064, 3064, 3064, 3063, 3063, 3429, 3063, 3056, 3056, 3056, 3056, 3056, 3057, 3057, 3057, 3057, 3055, 3055, 3055, 3055, 3055, 3058, 3058, 3058, 3058, 3058, 3058, 3058, 3060, 3060, 3060, 3060, 3060, 3046, 3046, 3043, 3043, 3043, 3046, 3046, 3044, 3044, 3044, 3043, 3089, 3095, 3095, 3095, 3088, 3099, 3099, 3113, 3113, 3090, 3095, 3096, 3096, 3096, 3097, 3775, 3775, 3099, 3099, 3754, 3097, 3097, 3097, 3763, 3762, 3762, 3762, 3778, 3099, 3759, 3761, 3761, 3761, 3760, 3099, 3099, 3097, 3755, 3091, 3091, 3083, 3754, 3754, 3754, 3754, 3754, 3076, 3076, 3076, 3075, 3075, 3754, 3082, 3752, 3074, 3074, 3753, 3753, 3064, 3757, 3757, 3757, 3778, 3757, 3757, 3757, 3757, 3757, 3750, 3755, 3755, 3147, 3147, 3147, 3103, 3103, 3103, 3104, 3104, 3104, 3104, 3104, 3104, 3124, 3124, 3124, 3124, 3124, 3124, 3124, 3124, 3124, 3124, 3126, 3126, 3103, 3146, 3122, 3122, 3122, 3122, 3122, 3122, 3122, 3123, 3123, 3123, 3123, 3101, 3101, 3101, 3101, 3101, 3102, 3102, 3102, 3127, 3127, 3127, 3127, 3127, 3127, 3153, 3155, 3156, 3156, 3180, 3180, 3156, 3178, 3179, 3787, 3154, 3156, 3152, 3152, 3152, 3152, 3105, 3105, 3108, 3108, 3108, 3109, 3109, 3109, 3109, 3109, 3109, 3111, 3114, 3106, 3106, 3107, 3107, 3107, 3107, 3113, 3113, 3113, 3134, 3115, 3131, 3134, 3153, 3136, 3136, 3136, 3136, 3136, 3136, 3136, 3135, 3135, 3137, 3137, 3114, 3134, 3134, 3134, 3135, 3135, 3134, 3133, 3134, 3134, 3115, 3130, 3130, 3130, 3130, 3130, 3130, 3128, 3128, 3129, 3129, 3129, 3129, 3128, 3128, 3125, 3125, 3125, 3125, 3151, 3151, 3151, 3131, 3131, 3132, 3132, 3132, 3127, 3131, 3131, 3131, 3127, 3133, 3133, 3133, 3133, 3160, 3160, 3160, 3116, 3116, 3137, 3140, 3765, 3138, 3796, 3796, 3159, 3160, 3777, 3156, 3158, 3777, 3139, 3139, 3799, 3779, 3779, 3779, 3779, 3779, 3779, 3777, 3777, 3770, 3770, 3775, 3139, 3782, 3778, 3786, 3786, 3797, 3797, 3797, 3770, 3770, 3770, 3777, 3777, 3139, 3791, 3791, 3791, 3766, 3139, 3156, 3782, 3799, 3723, 3159, 3799, 3793, 3793, 3767, 3777, 3804, 3788, 3788, 3797, 3797, 3797, 3797, 3797, 3799, 3799, 3787, 3787, 3139, 3139, 3139, 3789, 3795, 3795, 3795, 3775, 3775, 3792, 3797, 3833, 3785, 3139, 3139, 3139, 3799, 3799, 3799, 3799, 3799, 3799, 3139, 3115, 3775, 3775, 3797, 3139, 3139, 3770, 3193, 3193, 3193, 3193, 3193, 3186, 3186, 3186, 3186, 3186, 3186, 3187, 3187, 3192, 3192, 3192, 3185, 3188, 3188, 3188, 3190, 3191, 3189, 3807, 3782, 3782, 3782, 3809, 3809, 3810, 3810, 3782, 3981, 3981, 3808, 3808, 3815, 3815, 3984, 3978, 3978, 3981, 3782, 3781, 3781, 3781, 3814, 3981, 3809, 3814, 3814, 3783, 3783, 3783, 3783, 3807, 3981, 3815, 3981, 3981, 3891, 3984, 3894, 3816, 3812, 3159, 3816, 3984, 3781, 3781, 3812, 3812, 3987, 3809, 3810, 3810, 3810, 3815, 3977, 3813, 3813, 3814, 3894, 3806, 3977, 3978, 3978, 3977, 3977, 3977, 3977, 3977, 3977, 3977, 3177, 3802, 3177, 3803, 3976, 3977, 3975, 3805, 3804, 3805, 3980, 3977, 3977, 3977, 3806, 3156, 3912, 3980, 3980, 3175, 3175, 3175, 3175, 3175, 3175, 3175, 3175, 3175, 3173, 3173, 3174, 3174, 3174, 3174, 3174, 3171, 3171, 3172, 3975, 3175, 3201, 3199, 3199, 3199, 3199, 3199, 3199, 3199, 3199, 3199, 3199, 3200, 3200, 3199, 3199, 3910, 3977, 3198, 3198, 3198, 3977, 3977, 3911, 3204, 3204, 3165, 3165, 3165, 3162, 3145, 3161, 3161, 3161, 3162, 3162, 3163, 3185, 3185, 3185, 3163, 3163, 3204, 3163, 3163, 3204, 3204, 3183, 3195, 3195, 3196, 3195, 3197, 3196, 3196, 3192, 3192, 3192, 3169, 3169, 3169, 3172, 3172, 3172, 3196, 3202, 3190, 3194, 3194, 3194, 3189, 3189, 3189, 3194, 3195, 3195, 3195, 3197, 3197, 3195, 3147, 3147, 3125, 3148, 3148, 3168, 3168, 3168, 3168, 3168, 3150, 3150, 3150, 3150, 3166, 3166, 3800, 3149, 3149, 3149, 3149, 3149, 3170, 3170, 3170, 3170, 3170, 3170, 3170, 3168, 3166, 3166, 3167, 3167, 3150, 3150, 3150, 3942, 3940, 3936, 3936, 3938, 3938, 3938, 3931, 3931, 3931, 3931, 3930, 3930, 3930, 3930, 3930, 3934, 3934, 3934, 3934, 3944, 3944, 3944, 3939, 3939, 3939, 3939, 3941, 3941, 3936, 3941, 3943, 3943, 3941, 3918, 3919, 3919, 3915, 3915, 3912, 3912, 3912, 3913, 3913, 3936, 3936, 3926, 3926, 3926, 3911, 3939, 3939, 3939, 3939, 3929, 3929, 3920, 3928, 3916, 3926, 3926, 3926, 3933, 3916, 3937, 3937, 3916, 3927, 3927, 3915, 3915, 3915, 3143, 3143, 3146, 3146, 3144, 3144, 3144, 3144, 3145, 3145, 3145, 3145, 3145, 3145, 3181, 3141, 3141, 3142, 3181, 3021, 3020, 3022, 3012, 3023, 3037, 3023, 3037, 3030, 3037, 3021, 3036, 3036, 3038, 3033, 3038, 3036, 3036, 3042, 3021, 3021, 3021, 3021, 3020, 3020, 3020, 3020, 3020, 3037, 3038, 3038, 3038, 3018, 3028, 3028, 3025, 3025, 3025, 3012, 3028, 3015, 3015, 3028, 3018, 3015, 3016, 3016, 3019, 3019, 3011, 3012, 3012, 3032, 3011, 3012, 3012, 3013, 3013, 3013, 3338, 3023, 3023, 3023, 3023, 3023, 3023, 3023, 3338, 3338, 3338, 3338, 3338, 3338, 3037, 3337, 3337, 3338, 3337, 3337, 3024, 3024, 3340, 3335, 3335, 3023, 3335, 3335, 3335, 3335, 3337, 3029, 3029, 3040, 3042, 3032, 3032, 3034, 3040, 3040, 3040, 3041, 3041, 3040, 3031, 3031, 3033, 3033, 3039, 3042, 3041, 3041, 3032, 3029, 3029, 3028, 3026, 3024, 3030, 3030, 3029, 3029, 3030, 3030, 3030, 3030, 3027, 3027, 3024, 3030, 3338, 3211, 3211, 3024, 3024, 3340]

#https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe?rq=1#
#this is the function to get the postcode for data that do not have postcode
def libraryPostCode(col):
  locator = Nominatim(user_agent="locGeoCoder")
  add = str(col["address"])
  pc = add[len(add)-4:]
  pc_len = len(pc)
  
  if pc_len == 4:
    pc = pc
  else:
    #generate coords
    coords = str(col["latitude"])+","+str(col["longitude"])
    
    #get postcode and suburb
    full_add = dict(locator.reverse(coords).raw)
    pc = full_add.get("address").get("postcode")
    #coords = lib_df["latitude"].astype(str)+","+lib_df["longitude"].astype(str)
  return pc

#read libraries data
lib_df = pd.read_csv(lib_path, engine='python')

#read latest database to be updated
#db_df = pd.read_csv(db_path, engine='python')

#for csv files with separator is semi colon ";"
db_df = pd.read_csv(db_path, delimiter=";")


#drop empty rows
lib_df = lib_df.dropna(how="all").reset_index(drop=True)
lib_df_cols = ["LAT","LONG"]

#check if first row are column headers
i = 0
for col in lib_df.columns:
  if col in lib_df_cols:
    i += 1
  else:
    pass

#if not column header, find column header row    
if i != 2:
  #find columns with the that contains address, long and lat
  add = lib_df.columns[lib_df.eq("Address").any()][0]
  lon = lib_df.columns[lib_df.eq("LONG").any()][0]
  lat = lib_df.columns[lib_df.eq("LAT").any()][0]

  #get row index where the columns are found
  add_idx = lib_df.index[lib_df[add]=="Address"].tolist()[0]
  lon_idx = lib_df.index[lib_df[lon]=="LONG"].tolist()[0]
  lat_idx = lib_df.index[lib_df[lat]=="LAT"].tolist()[0]

  if add_idx == lon_idx == lat_idx:
    #update columns
    #https://stackoverflow.com/questions/36784908/how-to-convert-rows-into-list-using-pandas-python?rq=1
    lib_df.columns = lib_df[add_idx:add_idx+1].values.tolist()[0]
    lib_df = lib_df.drop(add_idx)

    #rename columns
    lib_df.rename(columns={'LAT':'latitude'}, inplace=True)
    lib_df.rename(columns={'LONG':'longitude'}, inplace=True)
    lib_df.rename(columns={'Address':'address'}, inplace=True)
    lib_df["address"] = lib_df["address"].str.replace("  "," ")
    lib_df.rename(columns={'Suburb/Town':'suburb'}, inplace=True)
    lib_df = lib_df.drop("Phone",axis = 1)

    #recompute postcode for rows with invalid postcode values
    lib_df["postcode"] = lib_df.apply(libraryPostCode, axis = 1)
    lib_df["postcode"] = pd.to_numeric(lib_df["postcode"])

    #filter if postcode is not within victoria
    lib_df = lib_df[lib_df.postcode.isin(melb_postcode)]


    #set default values
    lib_df['if_charger_working'] = 1
    lib_df['usb_a'] = 0
    lib_df['usb_c'] = 0 
    lib_df['micro_usb'] = 1
    lib_df['plug_only'] = 1
    lib_df['id'] = 0

    lib_df['establishment_type'] = "Library"
    lib_df['if_wifi'] = 1
    lib_df['if_bathroom'] = 1
    lib_df['access_type'] = "Entrance(s) have limited access via a small lip or a steep ramp, but alternative access available"
    lib_df['other_amenities'] = ""
    lib_df["station_name"] = lib_df["suburb"].apply(lambda x: x if x.find("Library") != -1 else x+" Library")
    lib_df["suburb"] = lib_df["suburb"].str.replace("Library","").str.replace("Public","")
    lib_df['if_24h'] = 0
    lib_df['star_rating'] = None
    lib_df['mon_open'] =  "9:00:00"	
    lib_df['mon_close'] = "17:00:00"
    lib_df['tue_open'] = "9:00:00"	
    lib_df['tue_close'] = "17:00:00"
    lib_df['wed_open'] = "9:00:00"	
    lib_df['wed_close'] = "17:00:00"
    lib_df['thu_open'] = "9:00:00"
    lib_df['thu_close'] = "17:00:00"
    lib_df['fri_open'] = "9:00:00"
    lib_df['fri_close'] = "17:00:00"
    lib_df['sat_open'] = "10:00:00"	
    lib_df['sat_close'] = "16:00:00"	
    lib_df['sun_open'] = "12:00:00"
    lib_df['sun_close'] = "16:00:00"
    lib_df = lib_df[cols_db]

    #this is to update existing libraries in the table
    exs_cols = ['station_name','longitude', 'latitude', 'address', 'suburb', 'postcode', 'if_charger_working', 'usb_a', 'usb_c', 'micro_usb', 'plug_only', 'establishment_type', 'if_wifi', 'if_bathroom', 'access_type', 'other_amenities', 'mon_open', 'mon_close', 'tue_open', 'tue_close', 'wed_open', 'wed_close', 'thu_open', 'thu_close', 'fri_open', 'fri_close', 'sat_open', 'sat_close', 'sun_open', 'sun_close', 'if_24h']
    exlib_df = lib_df[exs_cols]

    #https://stackoverflow.com/questions/24768657/replace-column-values-based-on-another-dataframe-python-pandas-better-way
    db_df = db_df.set_index('station_name')
    exlib_df = lib_df.set_index('station_name')
    db_df.update(exlib_df)
    db_df.reset_index(inplace=True)

    db_df['id'] = db_df['id'].astype(int)
    db_df = db_df[cols_db]
    db_list = db_df["station_name"].to_list()

    #get data from library in melbourne that is not in the database
    lib_df2 = lib_df[~lib_df.station_name.isin(db_list)].reset_index(drop=True)
    
    #get max values from database for id generation
    id_max = db_df['id'].max()
    lib_df2['id'] = lib_df2['id'].index+id_max+1

    #add the data into the database with id numbers
    db_df = db_df.append(lib_df2).reset_index(drop=True)

    #cast all data to int
    db_df['postcode'] = db_df['postcode'].astype(int)
    db_df['if_charger_working'] = db_df['if_charger_working'].astype(int)
    db_df['usb_a'] = db_df['usb_a'].astype(int)
    db_df['usb_c'] = db_df['usb_c'].astype(int)
    db_df['micro_usb'] = db_df['micro_usb'].astype(int)
    db_df['plug_only'] = db_df['plug_only'].astype(int)
    db_df['id'] = db_df['id'].astype(int)
    db_df['if_wifi'] = db_df['if_wifi'].astype(int)
    db_df['if_bathroom'] = db_df['if_bathroom'].astype(int)
    db_df['if_24h'] = db_df['if_24h'].astype(int)


    #create new updated database
    #this is to save the file as csv for additional external analysis in excel
    db_df.to_csv(r'new_db.csv', index = False)
    #mount the drive in google collabb

    #https://stackoverflow.com/questions/53898836/export-dataframe-as-csv-file-from-google-colab-to-google-drive
    #!cp maccas_24hrs_data.csv "/content/drive/My Drive/B3_FIT5120FIT5122/Iteration 2/Dataset/Python Code/"

  else:
    print("error, check uploaded data")


time: 2.03 s


In [4]:
db_df

,id,station_name,longitude,latitude,address,suburb,postcode,if_charger_working,usb_a,usb_c,micro_usb,plug_only,establishment_type,if_wifi,if_bathroom,access_type,other_amenities,star_rating,mon_open,mon_close,tue_open,tue_close,wed_open,wed_close,thu_open,thu_close,fri_open,fri_close,sat_open,sat_close,sun_open,sun_close,if_24h
0,1,Northern Pavilion,144.962,-37.7849,360 Royal Parade Carlton North 3052,Carlton North,3052,1,0,1,1,0,Community Space,2,2,All entrances have steps,NaN,0,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,1
1,2,Sydenham,144.952,-37.7964,10 Gatehouse Street Parkville 3052,Parkville,3052,1,0,0,1,0,Housing Support Services,2,1,Main entrance is step free or with ramps,NaN,0,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,0
2,3,Common Ground Supportive Housing,144.958,-37.8028,660-674 Elizabeth Street Carlton 3000,Carlton,3000,1,0,1,0,0,Housing Support Services,2,1,Main entrance is step free or with ramps,NaN,0,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,0
3,5,6th Melbourne Scout Hall,144.928,-37.7919,55 McCracken Street Kensington 3031,Kensington,3031,1,0,1,1,0,Community Space,2,2,Main entrance is step free or with ramps,NaN,0,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,0
4,6,Flemington Oval Pavilion,144.958,-37.7915,54 Flemington Road Parkville 3052,Parkville,3052,1,1,0,1,0,Community Space,2,2,Main entrance is step free or with ramps,NaN,0,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,09:00:00,17:00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,224,McDonald's,144.904,-37.7423,McDonald's Keilor Road Essendon North 3040,Essendon North,3040,1,0,1,0,1,Restaurant,1,1,NaN,NaN,0,00:01:00,23:59:00,00:01:00,23:59:00,00:01:00,23:59:00,00:01:00,23:59:00,00:01:00,23:59:00,00:01:00,23:59:00,00:01:00,23:59:00,1
223,225,McDonald's,144.994,-37.797,Hoddle Street Abbotsford 3067,Abbotsford,3067,1,0,0,0,1,Restaurant,1,1,NaN,NaN,0,00:01:00,23:59:00,00:01:00,23:59:00,00:01:00,23:59:00,00:01:00,23:59:00,00:01:00,23:59:00,00:01:00,23:59:00,00:01:00,23:59:00,1
224,226,Manningham e-Library Centre @ Warrandyte,145.206902,-37.74659,402 Warrandyte Rd Manningham e-Library Centre ...,Manningham e- Centre @ Warrandyte,3113,1,0,0,1,1,Library,1,1,Entrance(s) have limited access via a small li...,,None,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,10:00:00,16:00:00,12:00:00,16:00:00,0
225,227,State Library,144.9643151,-37.81005651,328 Swanston Street State 3000,State,3000,1,0,0,1,1,Library,1,1,Entrance(s) have limited access via a small li...,,None,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,10:00:00,16:00:00,12:00:00,16:00:00,0


time: 86.6 ms
